In [3]:
import sys
sys.path.append('../../../python-bits/phytonic-simulator/')
from qoilsimul import *

delta, eta, sigma, Sigma, tau, theta, phi, chi = sp.var('δ η σ Σ τ θ φ χ')

# Example: the three-mode discrete quantum Fourier transform
---

A three-mode quantum Fourier transform is a unitary transformation that coherently scatters three input modes amongst thee output modes with fixed phases.

# Part 1: The theory circuit

As a first step, we'd like to adapt the theory circuit from [Phys. Rev. Lett. 128, 160501 (2022)](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.128.160501), Fig. 1(c).

We'd like to look at three different combinations of two detectors in coincidence and verify:
- Each output state has an equal probability of success, and
- Each output state has a fixed phase.

In [33]:
experiment = Experiment()

ancilla_photon = Photon(pathModes=1, dof=['path'], state=co(p1))
input_photon = Photon(pathModes=1, dof=['path'], state=co(p1))
experiment.addPhotons(ancilla_photon, input_photon)

beam_splitter_1 = BS(r=1/sp.Rational(2))
beam_splitter_2 = BS(r=1/sp.Rational(3))
beam_splitter_3 = BS(r=1/sp.Rational(2))
phase_shifter = PhaseShifter(theta=3*sp.pi/2)

beam_splitter_1.i = [ancilla_photon.o[0], experiment.emptyMode()]
phase_shifter.i = [beam_splitter_1.o[0]]
beam_splitter_2.i = [input_photon.o[0], beam_splitter_1.o[1]]
beam_splitter_3.i = [beam_splitter_2.o[0], phase_shifter.o[0]]

list_of_elements = [beam_splitter_1, phase_shifter, beam_splitter_2, beam_splitter_3]
experiment.addElements(*list_of_elements)

detector = Detectors(numberOfDetectors=2)
detector.i = [*beam_splitter_3.o]
experiment.addDetectors(detector)

print('\n Input state:')
experiment.state

Experiment initialised.

 Input state:


x[a]*x[b]

In [34]:
experiment.generateCircuit()
print(experiment.circuit)

a : p -BS-PSh----BS---D
b : p --------BS-BS---D
c : ...BS-----BS-------



## First detector combination (detector 1 + detector 2)

In [35]:
experiment.build()
print('\nTwo-fold success probability: {}'.format(experiment.successProbability))

print('\nFinal state:')
experiment.postSelectedState


Two-fold success probability: 1/9

Final state:


x[a]*x[b]/3

## Second detector combination (detector 2 + detector 3)

In [31]:
experiment.addElements(*list_of_elements)
detector.i = [beam_splitter_3.o[0], beam_splitter_2.o[1]]
experiment.addDetectors(detector)

experiment.build()
print('\nTwo-fold success probability: {}'.format(experiment.successProbability))
print('\n Final state:')
experiment.postSelectedState


Two-fold success probability: 1/9

 Final state:


(-1/6 - sqrt(3)*I/6)*x[b]*x[c]

## Third detector combination (detector 1 + detector 3)

In [36]:
experiment.addElements(*list_of_elements)
detector.i = [beam_splitter_3.o[1], beam_splitter_2.o[1]]
experiment.addDetectors(detector)

experiment.build()
print('\nTwo-fold success probability: {}'.format(experiment.successProbability))
print('\n Final state:')
experiment.postSelectedState


Two-fold success probability: 1/9

 Final state:


(-1/6 + sqrt(3)*I/6)*x[a]*x[c]

# Part 2: The real experimental circuit

In the lab, we want to implement a polarisation-based version of the QFT circuit described above. However, we must again verify that, for three different combinations detectors:
- Each output state has an equal probability of success, and
- Each output state has a fixed phase.

In [140]:
experiment = Experiment()

photon_state = Photon(pathModes=1, dof=['path','pol'],
                state=co(p1,V)*co(p1,H))
experiment.addPhotons(photon_state)

bd1 = BD(pathmodes=1, walking_pol=H, label='BD1')
h1, h2 = HWP(theta=sp.pi/4,label='π/4'), HWP(theta=0,label='0')

h6 = HWP(theta=sp.pi/8,label='π/8')
bd4 = BD(pathmodes=3, walking_pol=H, label='BD4')
extra_ps = RelPhaseShifter(theta=sp.pi/2, label='∇')
rps = PhaseShifter(theta=sp.pi, label='∇')
h7, h8 = HWP(theta=-0.3078-sp.pi,label='0.3'), HWP(theta=-sp.pi/4,label='π/4')
bd5 = BD(pathmodes=2, walking_pol=H, label='BD5')

h9 = HWP(theta=sp.pi/8,label='π/8')
bd6 = BD(pathmodes=1, walking_pol=H, label='BD6')

bd1.i = [res.o[0], experiment.emptyMode()]
h1.i =  [bd1.o[1]]
h2.i =  [bd1.o[0]]
h6.i =  [res.o[0]]
extra_ps.i = [h6.o[0]]
bd4.i = [h6.o[0], h1.o[0], experiment.emptyMode()]
h7.i =  [bd4.o[1]]
h8.i =  [bd4.o[0]]
bd5.i = [h8.o[0], h7.o[0], experiment.emptyMode()]
rps.i = [bd5.o[0]]
h9.i =  [bd5.o[1]]
bd6.i = [h9.o[0], experiment.emptyMode()]

experiment.addElements(bd1,h1,h6,extra_ps,bd4,h7,h8,rps,bd5,h9,bd6)

detector = Detectors(numberOfDetectors=2)
detector.i = [*bd6.o]

experiment.addDetectors(detector)
experiment.generateCircuit()
print(experiment.circuit)
experiment.build()

Experiment initialised.
a : p -BD1-----π/8-∇-BD4-----π/4-∇-BD5------------
b : ...BD1-π/4-------BD4-0.3-------BD5-π/8-BD6---D
c : .................BD4--------------------------
d : ...............................BD5------------
e : .......................................BD6---D



## First detector combination (detector 1 + detector 2)

In [141]:
experiment.build()
print('\nTwo-fold success probability: {}'.format(experiment.successProbability))

print('\nFinal state:')
experiment.postSelectedState


Two-fold success probability: 0.111130007442022

Final state:


0.235722302129033*sqrt(2)*I*x[b, V]*x[e, H]

## Second detector combination (detector 2 + detector 3)

In [142]:
detector.i = [bd6.o[1], bd5.o[2]]
experiment.addDetectors(detector)

experiment.build()
print('\nTwo-fold success probability: {}'.format(experiment.successProbability))
print('\n Final state:')
experiment.postSelectedState


Two-fold success probability: 0.111101672593145

 Final state:


(0.166553250091243 + 0.288724241235801*I)*x[d, H]*x[e, H]

## Third detector combination (detector 1 + detector 3)

In [143]:
detector.i = [bd6.o[0], bd5.o[2]]
experiment.addDetectors(detector)

experiment.build()
print('\nTwo-fold success probability: {}'.format(experiment.successProbability))
print('\n Final state:')
experiment.postSelectedState


Two-fold success probability: 0.111101672593145

 Final state:


(-0.166553250091243 + 0.288724241235801*I)*x[b, V]*x[d, H]

which coincide with the theory circuit.